# Régression Linéaire De Zéro

Dans ce notebook, vous allez apprendre à implémenter la **Régression Linéaire** de zéro en utilisant uniquement NumPy. 

## Objectifs d'Apprentissage
- Comprendre les fondements mathématiques de la régression linéaire
- Implémenter la fonction de prédiction : $y = wx + b$
- Calculer l'erreur quadratique moyenne (MSE)
- Calculer les gradients en utilisant le calcul différentiel
- Implémenter l'optimisation par descente de gradient
- Entraîner un modèle pour prédire les notes des étudiants en fonction du temps d'étude
- Sauvegarder et charger les poids du modèle entraîné
- Faire des prédictions sur de nouvelles données

## Jeu de Données
Nous utiliserons un jeu de données sur les temps d'étude des étudiants (en minutes) et leurs notes correspondantes (0-100).

## Table des Matières
- [1 - Importer les Packages](#1)
- [2 - Charger et Visualiser le Jeu de Données](#2)
- [3 - Théorie de la Régression Linéaire](#3)
  - [3.1 - La Formule de Prédiction](#3-1)
  - [3.2 - Erreur Quadratique Moyenne (MSE)](#3-2)
  - [3.3 - Calcul des Gradients](#3-3)
  - [3.4 - Descente de Gradient](#3-4)
- [4 - Implémentation De Zéro](#4)
  - [4.1 - Fonction de Prédiction](#4-1)
  - [4.2 - Fonction de Perte MSE](#4-2)
  - [4.3 - Calcul des Gradients](#4-3)
  - [4.4 - Descente de Gradient](#4-4)
  - [4.5 - Fonction d'Entraînement](#4-5)
- [5 - Entraîner le Modèle](#5)
- [6 - Sauvegarder les Poids du Modèle](#6)
- [7 - Charger les Poids du Modèle](#7)
- [8 - Tester sur Nouvelles Données](#8)
- [9 - Visualiser les Résultats](#9)
- [10 - Conclusion](#10)

<a name='1'></a>
## 1 - Importer les Packages

Commençons par importer les packages nécessaires :
- **numpy** : Pour les calculs numériques
- **matplotlib** : Pour les visualisations
- **pandas** : Pour charger et manipuler le jeu de données
- **pickle** : Pour sauvegarder et charger les poids du modèle

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os

%matplotlib inline

<a name='2'></a>
## 2 - Charger et Visualiser le Jeu de Données

Chargeons notre jeu de données contenant les temps d'étude des étudiants et leurs notes correspondantes.

In [ ]:
# Charger le jeu de données
data = pd.read_csv('study_grades_dataset.csv')

# Afficher les premières lignes
print("Premiers 5 échantillons du jeu de données :")
print(data.head())

# Afficher les statistiques du jeu de données
print("\nStatistiques du Jeu de Données :")
print(data.describe())

print(f"\nNombre total d'échantillons : {len(data)}")

In [ ]:
# Extraire les caractéristiques (X) et les étiquettes (y)
X = data['study_time_minutes'].values
y = data['grade'].values

print(f"Forme de X : {X.shape}")
print(f"Forme de y : {y.shape}")

# Visualiser le jeu de données
plt.figure(figsize=(10, 6))
plt.scatter(X, y, alpha=0.6, color='blue', edgecolors='k')
plt.xlabel('Temps d\'Étude (minutes)', fontsize=12)
plt.ylabel('Note', fontsize=12)
plt.title('Notes des Étudiants vs Temps d\'Étude', fontsize=14)
plt.grid(True, alpha=0.3)
plt.show()

print("\nComme vous pouvez le constater, il y a une corrélation positive entre le temps d'étude et les notes !")

<a name='3'></a>
## 3 - Théorie de la Régression Linéaire

Avant d'implémenter, comprenons les mathématiques derrière la régression linéaire.

<a name='3-1'></a>
### 3.1 - La Formule de Prédiction

En régression linéaire, nous voulons trouver une relation linéaire entre la caractéristique d'entrée $x$ (temps d'étude) et la sortie $y$ (note).

La formule de prédiction est :

$$\hat{y} = wx + b$$

Où :
- $\hat{y}$ est la valeur prédite
- $x$ est la caractéristique d'entrée (temps d'étude)
- $w$ est le poids (pente de la droite)
- $b$ est le biais (ordonnée à l'origine)

Pour plusieurs échantillons, nous pouvons vectoriser ceci comme :

$$\hat{Y} = wX + b$$

Où $X$ et $\hat{Y}$ sont des vecteurs de toutes les caractéristiques d'entrée et prédictions.

<a name='3-2'></a>
### 3.2 - Erreur Quadratique Moyenne (MSE)

Pour entraîner notre modèle, nous devons mesurer à quel point nos prédictions sont éloignées des valeurs réelles. Nous utilisons **l'Erreur Quadratique Moyenne (MSE)** comme fonction de perte :

$$\text{MSE} = \frac{1}{m} \sum_{i=1}^{m} (\hat{y}^{(i)} - y^{(i)})^2$$

Où :
- $m$ est le nombre d'exemples d'entraînement
- $\hat{y}^{(i)}$ est la valeur prédite pour l'exemple $i$
- $y^{(i)}$ est la valeur réelle pour l'exemple $i$

La MSE mesure la différence quadratique moyenne entre les prédictions et les valeurs réelles. Notre objectif est de **minimiser cette erreur** en trouvant les meilleures valeurs pour $w$ et $b$.

<a name='3-3'></a>
### 3.3 - Calcul des Gradients

Pour minimiser la MSE, nous devons savoir comment la perte change lorsque nous ajustons $w$ et $b$. Cela se fait en utilisant les **gradients** (dérivées partielles).

Le gradient de la MSE par rapport à $w$ :

$$\frac{\partial \text{MSE}}{\partial w} = \frac{2}{m} \sum_{i=1}^{m} (\hat{y}^{(i)} - y^{(i)}) \cdot x^{(i)}$$

Sous forme vectorisée :

$$\frac{\partial \text{MSE}}{\partial w} = \frac{2}{m} X^T (\hat{Y} - Y)$$

Le gradient de la MSE par rapport à $b$ :

$$\frac{\partial \text{MSE}}{\partial b} = \frac{2}{m} \sum_{i=1}^{m} (\hat{y}^{(i)} - y^{(i)})$$

Sous forme vectorisée :

$$\frac{\partial \text{MSE}}{\partial b} = \frac{2}{m} \sum (\hat{Y} - Y)$$

Ces gradients nous indiquent la direction et l'amplitude pour ajuster $w$ et $b$ afin de réduire l'erreur.

<a name='3-4'></a>
### 3.4 - Descente de Gradient

La **Descente de Gradient** est un algorithme d'optimisation qui ajuste itérativement les paramètres pour minimiser la fonction de perte.

Les règles de mise à jour sont :

$$w := w - \alpha \frac{\partial \text{MSE}}{\partial w}$$

$$b := b - \alpha \frac{\partial \text{MSE}}{\partial b}$$

Où :
- $\alpha$ est le **taux d'apprentissage** (contrôle la taille du pas)
- Les gradients nous indiquent dans quelle direction nous déplacer

Nous répétons ce processus pendant de nombreuses itérations jusqu'à ce que le modèle converge (la perte cesse de diminuer significativement).

<a name='4'></a>
## 4 - Implémentation De Zéro

Maintenant implémentons chaque composant de la régression linéaire de zéro !

<a name='4-1'></a>
### 4.1 - Fonction de Prédiction

D'abord, implémentons la fonction de prédiction : $\hat{y} = wx + b$

In [ ]:
def predict(X, w, b):
    """
    Calculer les prédictions en utilisant la formule de régression linéaire.
    
    Arguments :
    X -- caractéristiques d'entrée, tableau numpy de forme (m,) où m est le nombre d'exemples
    w -- paramètre de poids (pente)
    b -- paramètre de biais (ordonnée à l'origine)
    
    Retourne :
    y_pred -- prédictions, tableau numpy de forme (m,)
    """
    y_pred = w * X + b
    return y_pred

In [ ]:
# Tester la fonction de prédiction
test_X = np.array([100, 200, 300])
test_w = 0.03
test_b = 40

test_predictions = predict(test_X, test_w, test_b)
print(f"Entrées de test : {test_X}")
print(f"Prédictions de test : {test_predictions}")
print(f"\nAvec w={test_w} et b={test_b} :")
print(f"  - {test_X[0]} minutes d'étude → note prédite : {test_predictions[0]}")
print(f"  - {test_X[1]} minutes d'étude → note prédite : {test_predictions[1]}")
print(f"  - {test_X[2]} minutes d'étude → note prédite : {test_predictions[2]}")

<a name='4-2'></a>
### 4.2 - Fonction de Perte MSE

Maintenant implémentons la fonction de perte d'Erreur Quadratique Moyenne.

In [ ]:
def compute_mse(y_true, y_pred):
    """
    Calculer l'Erreur Quadratique Moyenne entre les valeurs vraies et prédites.
    
    Arguments :
    y_true -- valeurs réelles, tableau numpy de forme (m,)
    y_pred -- valeurs prédites, tableau numpy de forme (m,)
    
    Retourne :
    mse -- erreur quadratique moyenne (scalaire)
    """
    m = len(y_true)
    mse = (1/m) * np.sum((y_pred - y_true)**2)
    return mse

In [ ]:
# Tester la fonction MSE
test_y_true = np.array([50, 60, 70])
test_y_pred = np.array([48, 62, 68])

test_mse = compute_mse(test_y_true, test_y_pred)
print(f"Valeurs vraies : {test_y_true}")
print(f"Valeurs prédites : {test_y_pred}")
print(f"Erreur Quadratique Moyenne : {test_mse:.2f}")
print(f"\nUne MSE plus faible signifie de meilleures prédictions !")

<a name='4-3'></a>
### 4.3 - Calcul des Gradients

Implémenter le calcul du gradient pour $w$ et $b$.

In [ ]:
def compute_gradients(X, y_true, y_pred):
    """
    Calculer les gradients de la MSE par rapport à w et b.
    
    Arguments :
    X -- caractéristiques d'entrée, tableau numpy de forme (m,)
    y_true -- valeurs réelles, tableau numpy de forme (m,)
    y_pred -- valeurs prédites, tableau numpy de forme (m,)
    
    Retourne :
    dw -- gradient par rapport à w (scalaire)
    db -- gradient par rapport à b (scalaire)
    """
    m = len(y_true)
    
    # Calculer l'erreur
    error = y_pred - y_true
    
    # Calculer les gradients
    dw = (2/m) * np.sum(error * X)
    db = (2/m) * np.sum(error)
    
    return dw, db

In [ ]:
# Tester la fonction de gradient
test_X = np.array([100, 200, 300])
test_y_true = np.array([50, 60, 70])
test_y_pred = np.array([48, 62, 68])

test_dw, test_db = compute_gradients(test_X, test_y_true, test_y_pred)
print(f"Gradient par rapport à w (dw) : {test_dw:.4f}")
print(f"Gradient par rapport à b (db) : {test_db:.4f}")
print(f"\nCes gradients nous indiquent comment ajuster w et b pour réduire l'erreur.")

<a name='4-4'></a>
### 4.4 - Descente de Gradient

Implémenter une étape de descente de gradient pour mettre à jour les paramètres.

In [ ]:
def gradient_descent_step(w, b, dw, db, learning_rate):
    """
    Effectuer une étape de descente de gradient.
    
    Arguments :
    w -- poids actuel
    b -- biais actuel
    dw -- gradient par rapport à w
    db -- gradient par rapport à b
    learning_rate -- taille du pas pour les mises à jour
    
    Retourne :
    w_new -- poids mis à jour
    b_new -- biais mis à jour
    """
    w_new = w - learning_rate * dw
    b_new = b - learning_rate * db
    
    return w_new, b_new

In [ ]:
# Tester l'étape de descente de gradient
test_w = 0.03
test_b = 40
test_dw = 0.5
test_db = 2.0
test_lr = 0.01

new_w, new_b = gradient_descent_step(test_w, test_b, test_dw, test_db, test_lr)
print(f"Avant mise à jour : w = {test_w}, b = {test_b}")
print(f"Gradients : dw = {test_dw}, db = {test_db}")
print(f"Taux d'apprentissage : {test_lr}")
print(f"Après mise à jour : w = {new_w}, b = {new_b}")

<a name='4-5'></a>
### 4.5 - Fonction d'Entraînement

Maintenant combinons tout dans une fonction d'entraînement complète.

In [ ]:
def train_linear_regression(X, y, learning_rate=0.0001, num_iterations=1000, print_cost=True):
    """
    Entraîner un modèle de régression linéaire en utilisant la descente de gradient.
    
    Arguments :
    X -- caractéristiques d'entrée, tableau numpy de forme (m,)
    y -- valeurs cibles, tableau numpy de forme (m,)
    learning_rate -- taux d'apprentissage pour la descente de gradient
    num_iterations -- nombre d'itérations d'entraînement
    print_cost -- si True, afficher le coût toutes les 100 itérations
    
    Retourne :
    w -- poids entraîné
    b -- biais entraîné
    costs -- liste des coûts pendant l'entraînement (pour le traçage)
    """
    # Initialiser les paramètres
    w = 0.0
    b = 0.0
    
    costs = []
    
    for i in range(num_iterations):
        # Passe avant : calculer les prédictions
        y_pred = predict(X, w, b)
        
        # Calculer la perte
        cost = compute_mse(y, y_pred)
        
        # Calculer les gradients
        dw, db = compute_gradients(X, y, y_pred)
        
        # Mettre à jour les paramètres
        w, b = gradient_descent_step(w, b, dw, db, learning_rate)
        
        # Enregistrer le coût
        if i % 100 == 0:
            costs.append(cost)
            if print_cost:
                print(f"Itération {i} : MSE = {cost:.2f}")
    
    # Afficher les résultats finaux
    if print_cost:
        print(f"\n{'='*50}")
        print(f"Entraînement Terminé !")
        print(f"MSE Finale : {cost:.2f}")
        print(f"Paramètres appris : w = {w:.6f}, b = {b:.6f}")
        print(f"{'='*50}")
    
    return w, b, costs

<a name='5'></a>
## 5 - Entraîner le Modèle

Entraînons notre modèle de régression linéaire sur le jeu de données temps d'étude et notes !

In [ ]:
# Entraîner le modèle
print("Début de l'entraînement...\n")

w_trained, b_trained, training_costs = train_linear_regression(
    X, y,
    learning_rate=0.00001,
    num_iterations=2000,
    print_cost=True
)

In [ ]:
# Tracer la courbe d'entraînement
plt.figure(figsize=(10, 6))
plt.plot(range(0, len(training_costs)*100, 100), training_costs, linewidth=2, color='red')
plt.xlabel('Itération', fontsize=12)
plt.ylabel('Erreur Quadratique Moyenne (MSE)', fontsize=12)
plt.title('Perte d\'Entraînement au Fil du Temps', fontsize=14)
plt.grid(True, alpha=0.3)
plt.show()

print("La diminution de la perte montre que notre modèle apprend !")

<a name='6'></a>
## 6 - Sauvegarder les Poids du Modèle

Maintenant que nous avons entraîné notre modèle, sauvegardons les poids pour pouvoir les réutiliser plus tard sans réentraîner.

In [ ]:
# Créer le dossier model s'il n'existe pas
if not os.path.exists('model'):
    os.makedirs('model')
    print("Dossier 'model' créé.")

# Sauvegarder les poids entraînés
model_weights = {
    'w': w_trained,
    'b': b_trained
}

with open('model/linear_regression_weights.pkl', 'wb') as f:
    pickle.dump(model_weights, f)

print(f"\nPoids du modèle sauvegardés avec succès !")
print(f"Fichier : model/linear_regression_weights.pkl")
print(f"Poids : w = {w_trained:.6f}, b = {b_trained:.6f}")

<a name='7'></a>
## 7 - Charger les Poids du Modèle

Démontrons comment charger les poids sauvegardés. C'est utile lorsque vous voulez faire des prédictions sans réentraîner.

In [ ]:
# Charger les poids sauvegardés
with open('model/linear_regression_weights.pkl', 'rb') as f:
    loaded_weights = pickle.load(f)

w_loaded = loaded_weights['w']
b_loaded = loaded_weights['b']

print("Poids du modèle chargés avec succès !")
print(f"Poids chargés : w = {w_loaded:.6f}, b = {b_loaded:.6f}")

# Vérifier qu'ils correspondent aux poids entraînés
print(f"\nVérification :")
print(f"  w correspond : {np.isclose(w_loaded, w_trained)}")
print(f"  b correspond : {np.isclose(b_loaded, b_trained)}")

<a name='8'></a>
## 8 - Tester sur Nouvelles Données

Maintenant utilisons notre modèle entraîné pour faire des prédictions sur de nouveaux temps d'étude non vus !

In [ ]:
# Générer des temps d'étude de test aléatoires
np.random.seed(42)
test_study_times = np.random.randint(5, 3000, size=10)
test_study_times = np.sort(test_study_times)

# Faire des prédictions en utilisant les poids chargés
test_predictions = predict(test_study_times, w_loaded, b_loaded)

# Afficher les résultats
print("Prédictions sur Nouvelles Données :")
print("=" * 50)
for study_time, grade in zip(test_study_times, test_predictions):
    print(f"Temps d'Étude : {study_time:4d} minutes → Note Prédite : {grade:.1f}")
print("=" * 50)

In [ ]:
# Fonction de prédiction interactive
def predict_grade(study_minutes):
    """
    Prédire la note pour un temps d'étude donné.
    
    Arguments :
    study_minutes -- temps d'étude en minutes
    
    Retourne :
    predicted_grade -- note prédite (limitée à 0-100)
    """
    grade = predict(np.array([study_minutes]), w_loaded, b_loaded)[0]
    # Limiter à la plage de notes valide
    grade = np.clip(grade, 0, 100)
    return grade

# Essayer quelques exemples
print("\nPrédictions Interactives :")
print("=" * 50)
example_times = [60, 300, 600, 1200, 2400]
for time in example_times:
    predicted = predict_grade(time)
    print(f"Si vous étudiez {time:4d} minutes, note prédite : {predicted:.1f}")
print("=" * 50)

print("\n💡 Astuce : Vous pouvez modifier example_times pour tester vos propres durées d'étude !")

<a name='9'></a>
## 9 - Visualiser les Résultats

Visualisons à quel point notre modèle s'ajuste aux données.

In [ ]:
# Générer des prédictions pour l'ensemble du jeu de données
y_pred_all = predict(X, w_loaded, b_loaded)

# Créer la visualisation
plt.figure(figsize=(12, 6))

# Tracer les points de données originaux
plt.scatter(X, y, alpha=0.5, color='blue', label='Données Réelles', edgecolors='k')

# Tracer la droite de régression
plt.plot(X, y_pred_all, color='red', linewidth=2, label='Ajustement de Régression Linéaire')

# Tracer les prédictions de test
plt.scatter(test_study_times, test_predictions, color='green', 
            s=100, marker='*', label='Prédictions de Test', 
            edgecolors='black', linewidth=1.5)

plt.xlabel('Temps d\'Étude (minutes)', fontsize=12)
plt.ylabel('Note', fontsize=12)
plt.title('Régression Linéaire : Temps d\'Étude vs Note', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.show()

print(f"\nLa ligne rouge montre notre modèle appris : Note = {w_loaded:.6f} × Temps_Étude + {b_loaded:.6f}")

In [ ]:
# Calculer les métriques de performance finales
final_mse = compute_mse(y, y_pred_all)
rmse = np.sqrt(final_mse)

# Calculer le R-carré (coefficient de détermination)
ss_res = np.sum((y - y_pred_all)**2)  # Somme des carrés des résidus
ss_tot = np.sum((y - np.mean(y))**2)  # Somme totale des carrés
r_squared = 1 - (ss_res / ss_tot)

print("Métriques de Performance du Modèle :")
print("=" * 50)
print(f"Erreur Quadratique Moyenne (MSE) : {final_mse:.2f}")
print(f"Racine de l'Erreur Quadratique Moyenne (RMSE) : {rmse:.2f}")
print(f"R-carré (R²) : {r_squared:.4f}")
print("=" * 50)
print(f"\n📊 R² = {r_squared:.4f} signifie que notre modèle explique {r_squared*100:.2f}% de la variance des données !")

<a name='10'></a>
## 10 - Conclusion

Félicitations ! 🎉 Vous avez implémenté avec succès la régression linéaire de zéro !

### Ce que vous avez appris :

1. **Fondements Mathématiques :**
   - Formule de prédiction : $\hat{y} = wx + b$
   - Fonction de perte Erreur Quadratique Moyenne (MSE)
   - Calcul du gradient en utilisant le calcul différentiel
   - Optimisation par descente de gradient

2. **Compétences d'Implémentation :**
   - Construit tous les composants de zéro en utilisant NumPy
   - Implémenté la passe avant (prédictions)
   - Implémenté la passe arrière (gradients)
   - Créé une boucle d'entraînement complète

3. **Flux de Travail ML Pratique :**
   - Chargé et visualisé les données
   - Entraîné un modèle avec descente de gradient
   - Sauvegardé les poids du modèle pour réutilisation
   - Chargé les poids et fait des prédictions
   - Évalué la performance du modèle

### Insights Clés :

- La régression linéaire trouve la meilleure ligne droite à travers les données
- La descente de gradient améliore itérativement le modèle en suivant le gradient négatif
- Le taux d'apprentissage contrôle la vitesse à laquelle le modèle apprend
- La MSE mesure à quel point les prédictions sont proches des valeurs réelles
- Les poids du modèle peuvent être sauvegardés et réutilisés sans réentraîner

### Prochaines Étapes :

- Essayer différents taux d'apprentissage et voir comment ils affectent l'entraînement
- Expérimenter avec différents nombres d'itérations
- Ajouter la normalisation des données pour améliorer l'entraînement
- Essayer la régression polynomiale pour les relations non linéaires
- Implémenter la régularisation pour éviter le surapprentissage

### Rappel :

C'est la base pour comprendre des modèles plus complexes comme les réseaux de neurones ! Les concepts que vous avez appris ici (passe avant, calcul de perte, gradients, optimisation) sont les mêmes utilisés en apprentissage profond.

Continuez à pratiquer ! 🚀

---

## Bonus : Dérivation Mathématique

Pour ceux intéressés par les mathématiques, voici comment nous dérivons les gradients :

### Perte MSE :
$$L = \frac{1}{m} \sum_{i=1}^{m} (\hat{y}^{(i)} - y^{(i)})^2$$

### Gradient par rapport à w :
$$\frac{\partial L}{\partial w} = \frac{\partial}{\partial w} \left[ \frac{1}{m} \sum_{i=1}^{m} (wx^{(i)} + b - y^{(i)})^2 \right]$$

En utilisant la règle de la chaîne :
$$\frac{\partial L}{\partial w} = \frac{2}{m} \sum_{i=1}^{m} (wx^{(i)} + b - y^{(i)}) \cdot x^{(i)}$$

### Gradient par rapport à b :
$$\frac{\partial L}{\partial b} = \frac{\partial}{\partial b} \left[ \frac{1}{m} \sum_{i=1}^{m} (wx^{(i)} + b - y^{(i)})^2 \right]$$

En utilisant la règle de la chaîne :
$$\frac{\partial L}{\partial b} = \frac{2}{m} \sum_{i=1}^{m} (wx^{(i)} + b - y^{(i)})$$

Ce sont les formules que nous avons implémentées dans notre fonction `compute_gradients` !